In [3]:
import torch
import torchvision.models as models
import torch.nn as nn
import math

models_to_analyze = {
    'alexnet': {'input_size': 227, 'model_loader': models.alexnet},
    'resnet50': {'input_size': 224, 'model_loader': models.resnet50},
    'mobilenet_v2': {'input_size': 224, 'model_loader': models.mobilenet_v2},
}

N = 4 # Batch Size

In [9]:
def calculate_density(tensor):
    if tensor.numel() == 0:
        return 0.0
    non_zeros = torch.count_nonzero(tensor)
    density = non_zeros.float() / tensor.numel()
    return density.item()

def calculate_output_dim(input_dim, kernel_size, stride, padding, dilation):
    if isinstance(kernel_size, int): kernel_size = (kernel_size, kernel_size)
    if isinstance(stride, int): stride = (stride, stride)
    if isinstance(padding, int): padding = (padding, padding)
    if isinstance(dilation, int): dilation = (dilation, dilation)

    output_h = math.floor(((input_dim[0] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0]) + 1)
    output_w = math.floor(((input_dim[1] + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1]) + 1)
    return output_h, output_w

In [11]:
device = torch.device("cpu")

results = {}

for model_name, config in models_to_analyze.items():
    print(f"--- Analyzing {model_name} ---")

    model = config['model_loader'](pretrained=True).to(device)
    model.eval()

    first_conv_layer = None
    activation_layer = None
    batch_norm_layer = None

    if model_name == 'alexnet':
        first_conv_layer = model.features[0]
        activation_layer = model.features[1]
        C = 3
    elif model_name == 'resnet50':
        first_conv_layer = model.conv1
        batch_norm_layer = model.bn1
        activation_layer = model.relu
        C = first_conv_layer.in_channels
    elif model_name == 'mobilenet_v2':
        first_conv_layer = model.features[0][0]
        batch_norm_layer = model.features[0][1]
        activation_layer = model.features[0][2]
        C = first_conv_layer.in_channels
    else:
        print(f"Don't know how to find the layers for {model_name}. Skipping.")
        continue

    if not isinstance(first_conv_layer, nn.Conv2d):
         print(f"Identified first layer for {model_name} is not nn.Conv2d. Skipping.")
         continue
    if activation_layer is None:
         print(f"Could not find activation layer for {model_name}. Skipping.")
         continue


    M = first_conv_layer.out_channels
    kernel_size = first_conv_layer.kernel_size
    R, S = kernel_size
    stride = first_conv_layer.stride
    Hstride, Wstride = stride
    padding = first_conv_layer.padding
    dilation = first_conv_layer.dilation
    Hdilation, Wdilation = dilation
    weights = first_conv_layer.weight.data

    input_height = config['input_size']
    input_width = config['input_size']
    dummy_input = torch.randn(N, C, input_height, input_width).to(device)

    output_activations = None
    with torch.no_grad():
        x = first_conv_layer(dummy_input)
        if batch_norm_layer:
             x = batch_norm_layer(x)
        output_activations = activation_layer(x)

    input_density = calculate_density(dummy_input)
    weight_density = calculate_density(weights)
    output_density = calculate_density(output_activations)

    Q, P = calculate_output_dim((input_height, input_width), kernel_size, stride, padding, dilation)

    layer_info = {
        'Layer Type': type(first_conv_layer).__name__,
        'Activation Type': type(activation_layer).__name__,
        'Input Channels (C)': C,
        'Output Channels (M)': M,
        'Kernel Size (R, S)': kernel_size,
        'Stride (Hstride, Wstride)': stride,
        'Padding': padding,
        'Dilation (Hdilation, Wdilation)': dilation,
        'Input Tensor Shape': tuple(dummy_input.shape),
        'Weight Tensor Shape': tuple(weights.shape),
        'Output Tensor Shape (Post-Activation)': tuple(output_activations.shape),
        'Calculated Output Dim (Q, P)': (Q, P),
        'Input Density': f"{input_density:.6f}",
        'Weight Density': f"{weight_density:.6f}",
        'Output Activation Density (Post-Activation)': f"{output_density:.6f}",
    }
    results[model_name] = layer_info

    print(f"  Layer: {layer_info['Layer Type']}, Activation: {layer_info['Activation Type']}")
    print(f"  Parameters:")
    print(f"    C: {C}, M: {M}, R: {R}, S: {S}")
    print(f"    N: {N}, P (expected): {P}, Q (expected): {Q}")
    print(f"    Hstride: {Hstride}, Wstride: {Wstride}")
    print(f"    Padding: {padding}")
    print(f"    Dilation: {dilation}")
    print(f"  Tensor Shapes:")
    print(f"    Input: {layer_info['Input Tensor Shape']}")
    print(f"    Weights: {layer_info['Weight Tensor Shape']}")
    print(f"    Output (Post-Activation): {layer_info['Output Tensor Shape (Post-Activation)']}")
    print(f"  Densities:")
    print(f"    Input:   {layer_info['Input Density']}")
    print(f"    Weights: {layer_info['Weight Density']}")
    print(f"    Output (Post-Activation):  {layer_info['Output Activation Density (Post-Activation)']}")
    print("-" * 30)

--- Analyzing alexnet ---
  Layer: Conv2d, Activation: ReLU
  Parameters:
    C: 3, M: 64, R: 11, S: 11
    N: 4, P (expected): 56, Q (expected): 56
    Hstride: 4, Wstride: 4
    Padding: (2, 2)
    Dilation: (1, 1)
  Tensor Shapes:
    Input: (4, 3, 227, 227)
    Weights: (64, 3, 11, 11)
    Output (Post-Activation): (4, 64, 56, 56)
  Densities:
    Input:   1.000000
    Weights: 1.000000
    Output (Post-Activation):  0.418981
------------------------------
--- Analyzing resnet50 ---
  Layer: Conv2d, Activation: ReLU
  Parameters:
    C: 3, M: 64, R: 7, S: 7
    N: 4, P (expected): 112, Q (expected): 112
    Hstride: 2, Wstride: 2
    Padding: (3, 3)
    Dilation: (1, 1)
  Tensor Shapes:
    Input: (4, 3, 224, 224)
    Weights: (64, 3, 7, 7)
    Output (Post-Activation): (4, 64, 112, 112)
  Densities:
    Input:   1.000000
    Weights: 1.000000
    Output (Post-Activation):  0.657259
------------------------------
--- Analyzing mobilenet_v2 ---
  Layer: Conv2d, Activation: ReLU6
  P